In [ ]:
from selenium import webdriver
#driver=webdriver.Chrome(r'C:\Users\Meteora\anaconda3\Lib\site-packages\webdriver_manager\chromedriver.exe')

#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--no-sandbox')
#chrome = webdriver.Chrome(r'C:\Users\Meteora\anaconda3\Lib\site-packages\webdriver_manager\chromedriver', chrome_options=chrome_options)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#driver.get("https://www.google.com")
url='https://www.amazon.com'
driver.get(url)

In [ ]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup

s=HTMLSession()
url = 'https://www.amazon.in/s?k=masks&crid=2UVWHSIMTSKVQ&sprefix=masks%2Caps%2C253&ref=nb_sb_noss_1'
#url = 'https://www.amazon.in/s?k=DEo&i=beauty&rh=n%3A1355016031%2Cp_89%3AKELYN&s=price-desc-rank&dc&crid=3K4SMLF0ONZJD&qid=1650604503&rnid=3837712031&sprefix=deo%2Caps%2C185&ref=sr_st_price-desc-rank'
#url = 'https://www.boddess.com/makeup/eyes/eyebrow-pencils-enhancers?manufacturer=56'
def getdata(url):
    r=s.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup


#print(getdata(url))
#print(soup)

In [ ]:
def getnextpage(soup):
    page=soup.find('ul',{'class': 'a-pagination'})
    print(page)
    if not page.find('li',{'class': 'a-disabled a-last'}):
        url = 'http://www.amazon.in' + str(page.find('li',{'class':'a-last'}).find('a')['href'])
        #url = 'https://www.boddess.com' + str(page.find('li',{'class':'a-last'}).find('a')['href'])
        print("found")
        return url
    else:
        print("Not found")
        return
    
    soup=getdata(url)
    
#print(getnextpage(soup))
    
soup=getdata(url)
results=soup.find_all('div',{'data-component-type':'s-search-result'})
#results=soup.find_all('div',{'class':'product details product-item-details'})
len(results)
#print(results)

In [ ]:
item=results[3]
atag=item.h2.a
#atag=item.strong.a
desc=atag.text.strip()
print(desc)

In [ ]:
atag.get('href')

In [ ]:
url='http://www.amazon.com' + atag.get('href')
url=atag.get('href')
print(url)

In [ ]:

price_parent= item.find('span','a-offscreen').text
#price_parent= item.find('span','data-price-amount').text
print(price_parent)

'''
price_parent= item.find('div','price-box price-final_price').text
#price_parent= price_parent.find('span','data-price-amount').text
print(price_parent)
'''

In [ ]:
'''
rating=item.i.text
print(rating)
'''

In [ ]:
def extractRecord(item):
    atag=item.h2.a
    description=atag.text.rstrip()
    url='http://www.amazon.com' + atag.get('href')
    try:
        price= item.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating=item.i.text
        review_count=item.find('span',{'class':'a-size-base','dir':'auto'}).text
    
    except AttributeError:
        rating=''
        review_count=''
        
    result=(description,price,rating,review_count,url)
    return result

#------------------------------------------------------------------------------------------#
'''
def extractRecord1(item):
    atag=item.strong.a
    description=atag.text.rstrip()
    url=atag.get('href')
    try:
        price_parent= item.find('div','price-box price-final_price').text
    except AttributeError:
        return
        
    result=(description,price_parent,url)
    return result
'''

In [ ]:
records=[]
results=soup.find_all('div',{'data-component-type':'s-search-result'})
#results=soup.find_all('div',{'class':'product details product-item-details'})

for item in results:
    record=extractRecord(item)
    if record:
        records.append(extractRecord(item))

In [ ]:
print(records)

In [ ]:
for i in records:
    print (i[0] + '---->' + i[1] + "\n")

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import csv

def get_url(search_term):
    template='https://www.amazon.in/s?k={}&ref=nb_sb_noss_1'
    #tmp='https://www.amazon.in/s?k=masks&rh=n%3A1350384031%2Cp_89%3A{}&dc&crid=2UVWHSIMTSKVQ&qid=1650617640&rnid=3837712031&sprefix=masks%2Caps%2C253&ref=sr_nr_p_89_2'
    search_term=search_term.replace(' ','+')
    
    #adding query
    #print("Printing templetate fomrat " , template.format(search_term)
    url = template.format(search_term)
    url += '&page={}'
    
    return url

def extractRecord(item):
    atag=item.h2.a
    description=atag.text.rstrip()
    url='http://www.amazon.in' + atag.get('href')
    try:
        price= item.find('span','a-offscreen').text
    except AttributeError:
        return
    
    try:
        rating=item.i.text
        review_count=item.find('span',{'class':'a-size-base','dir':'auto'}).text
    
    except AttributeError:
        rating=''
        review_count=''
        
    result=(description,price,rating,review_count,url)
    return result

def main(search_term):
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    #url='https://www.amazon.com'
    #driver.get(url)

    records = []
    url=get_url(search_term)

    
    #'''    Fetching all brand list 
    brand_list=[]
    driver.get(url.format(1))
    soup_filter = BeautifulSoup(driver.page_source, 'html.parser')
    results_brand=soup_filter.find('div',{'id':'brandsRefinements'})
    result_ul=results_brand.find('ul',{'aria-labelledby':'p_89-title'}).findAll("li", recursive=True)
    #print(result_ul)
    for i in result_ul:
        #print(i)
        #break
        #final_result=i.find('aria-label').text.rstrip()
        try:
            final_result=i['aria-label']
        except KeyError :
            final_result=''

        brand_list.append(final_result)
    
    #print(brand_list)    #checking the brand list 
    #'''    ------------------------
    
    for page in range(1,300):
        driver.get(url.format(page))
        #s=HTMLSession()
        #r=s.get(url)
        #print(r)
        #soup = BeautifulSoup(r.text, 'html.parser')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results=soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record = extractRecord(item)
            if record:
                records.append(record)
            #print(records)    #checking the iteration of the lists appended
    driver.close()       
    
    #df = pd.DataFrame({'col':records})
    df = pd.DataFrame(records,columns =['Description', 'Price', 'Rating', 'review_count', 'url'])
    print (df.head)
    # DataFrame operations -------------------------------------------------------------#
    df['Price']=df['Price'].replace('\₹','',regex=True)
    df['Price']=df['Price'].replace(',','',regex=True)
    df['Price'] = df['Price'].astype(float)
    max=df['Price'].max()
    print("obtained max output", df.loc[df['Price'] == df['Price'].max()])
    print("obtained min output", df.loc[df['Price'] == df['Price'].min()])
    print("Max price is : " , df['Price'].max(), "  |  " ,  "Min price is :" , df['Price'].min())
    # DataFrame operations -------------------------------------------------------------#
    
    
    with open(r'C:\Users\Meteora\Downloads\results.csv', 'w' , newline='', encoding='utf-8') as f:
        print("Writing records to file")
        writer=csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'review_count', 'url'])
        writer.writerows(records)
    
    df.to_csv(r'C:\Users\Meteora\Downloads\results_df.csv')
    print("Filed saved successfully")
    

In [29]:
main('perfume')



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Meteora\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


<bound method NDFrame.head of                                             Description Price Rating  \
0     Ustraa Base Camp Cologne - 100 ml - Perfume fo...  ₹607          
1       Ustraa Scuba Cologne - 100 ml - Perfume for Men  ₹607          
2     The Man Company Black EDT Perfume For Men - 50...  ₹389          
3     PureSense Desire Madagascar Vanilla Body Mist ...  ₹318          
4     Villain Perfume For Men 100 Ml - Eau De Parfum...  ₹540          
...                                                 ...   ...    ...   
3819  Bryan & Candy NewYork Nottie Berries Fragrance...  ₹560          
3820  Beardo Body Spray Gift Set For Men (Pack of 3)...  ₹749          
3821  Body Cupid Mr Wonderful Perfume for Men - Eau ...  ₹547          
3822  Perfumer's Club "Best Fragrance for Men 2020" ...  ₹998          
3823  All Good Scents CHIC Eau De Toilette, Perfume ...  ₹699          

     review_count                                                url  
0                  http://www.amaz

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Meteora\\Downloads\\results_df.csv'